In [1]:
import pandas as pd

In [2]:
# Specify the file path
file_path = r'D:\OneDrive\OneDrive - Rowad Modern Engineering\x004 Data Science\03.rme.db\00.repo\rme.db\10.visualize\01.bridges.cash\00.main.file\Bridges Sector Cash In And Out Data.xlsx'

In [3]:
# Read the "In Values" sheet into a DataFrame
in_values_df = pd.read_excel(file_path, sheet_name='In Values')

# Read the "Out Values" sheet into another DataFrame
out_values_df = pd.read_excel(file_path, sheet_name='Out Values')

In [4]:
# Merge the "In Values" and "Out Values" DataFrames on the "Project Name" column
merged_df = pd.merge(in_values_df, out_values_df, on='Project Name', suffixes=('_in', '_out'), how='outer')

# Fill NaN values with 0 for cash in and cash out columns
merged_df['Amount_in'].fillna(0, inplace=True)
merged_df['Amount_out'].fillna(0, inplace=True)

In [5]:
# Calculate cumulative cash flow for each project
merged_df['Cumulative Cash Flow'] = merged_df.groupby('Project Name')['Amount_in'].cumsum() - merged_df.groupby('Project Name')['Amount_out'].cumsum()

In [6]:
import pandas as pd
from bokeh.plotting import figure, show
from bokeh.models import ColumnDataSource, Select
from bokeh.layouts import column
from bokeh.io import curdoc, output_file
from bokeh.transform import factor_cmap
from bokeh.palettes import Viridis256  # Using Viridis256 for a large number of colors

# Convert the 'Date' column to datetime type if it's not already
in_values_df['Date'] = pd.to_datetime(in_values_df['Date'])

# Extract year and month from the 'Date' column
in_values_df['Year'] = in_values_df['Date'].dt.year
in_values_df['Month'] = in_values_df['Date'].dt.month_name()

# Group by year and month and calculate the sum of cash in for each month
cash_in_per_month = in_values_df.groupby(['Year', 'Month'])['Amount'].sum().reset_index()

# Create a new column with the combined year and month values
cash_in_per_month['YearMonth'] = cash_in_per_month.apply(lambda x: f"{x['Month']} {x['Year']}", axis=1)

# Create a Bokeh data source from the aggregated DataFrame
source = ColumnDataSource(data=cash_in_per_month)

# Define the Bokeh figure
p = figure(x_range=cash_in_per_month['YearMonth'], height=350, title="Cash In per Month")

# Create a bar chart for cash in with dynamically generated colors
unique_factors = cash_in_per_month['YearMonth'].unique()
num_factors = len(unique_factors)
color_mapper = factor_cmap(field_name='YearMonth', palette=Viridis256[:num_factors], factors=unique_factors)
p.vbar(x='YearMonth', top='Amount', source=source, width=0.8, color=color_mapper)

# Configure the plot
p.xaxis.major_label_orientation = 45  # Rotate x-axis labels for readability

# Create a Select widget to choose a project
project_select = Select(title="Select Project:", options=in_values_df['Project Name'].unique().tolist())

# Define a callback function to update the plot based on the selected project
def update_plot(attrname, old_value, new_value):
    selected_project = project_select.value
    new_data = cash_in_per_month[cash_in_per_month['Project Name'] == selected_project]
    source.data = ColumnDataSource(data=new_data).data

project_select.on_change('value', update_plot)

# Create a layout for the plot and Select widget
layout = column(project_select, p)

# Save the plot as an HTML file (optional)
output_file("cash_flow_plot.html")

# Show the plot
show(layout)


You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html



In [7]:
# Show the plot in the notebook
show(layout, notebook_handle=True)

You are generating standalone HTML/JS output, but trying to use real Python
callbacks (i.e. with on_change or on_event). This combination cannot work.

Only JavaScript callbacks may be used with standalone output. For more
information on JavaScript callbacks with Bokeh, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/interaction/callbacks.html

Alternatively, to use real Python callbacks, a Bokeh server application may
be used. For more information on building and running Bokeh applications, see:

    https://docs.bokeh.org/en/latest/docs/user_guide/server.html

